In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from torch.nn.functional import softmax
import nltk
from nltk.corpus import stopwords

C:\Users\cmarc\AppData\Local\Temp\ipykernel_3964\722912215.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\cmarc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cmarc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cmarc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Carregar o modelo pré-treinado e o tokenizador
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased')
# Carregar stop words em português
stop_words_portuguese = set(stopwords.words('portuguese'))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Função para remover stopwords em português
def remover_stopwords(texto):
    palavras = [palavra for palavra in texto.split() if palavra.lower() not in stop_words_portuguese]
    return ' '.join(palavras)


In [5]:
# Função para realizar a análise de sentimentos
def analisar_sentimento(texto):
    if not texto or all(t.strip() == '' for t in texto.split()):
        # Caso o texto esteja vazio ou seja uma lista de textos vazios, retornar valores padrão
        return 'Indefinido', [0.5, 0.5]  # Sentimento indefinido, probabilidade neutra

    # Tokenizar o texto e converter para tensores
    tokens = tokenizer.encode(texto, return_tensors='pt')

    # Passar os tokens pelo modelo
    with torch.no_grad():
        outputs = model(tokens)

    # Obter probabilidades de cada classe (positiva e negativa)
    probabilidade = softmax(outputs.logits, dim=1)
    
    # Classificar como positivo ou negativo com base na probabilidade
    sentimento = "Positivo" if probabilidade[0][1] > probabilidade[0][0] else "Negativo"

    return sentimento, probabilidade.tolist()[0]  # Convertendo para lista


In [6]:
# Função para extrair os cinco tópicos mais comuns
def extrair_topicos_comuns(comentario):
    # Utilizar TF-IDF para vetorização, incluindo stop words em inglês
    vectorizer = TfidfVectorizer(max_features=1000, dtype=np.float32)
    X = vectorizer.fit_transform(comentario)

    # Aplicar a decomposição de matriz não-negativa (NMF)
    nmf = NMF(n_components=5, random_state=1)
    nmf.fit(X)

    # Obter as palavras-chave mais comuns para cada tópico
    palavras_chave = []
    for i, topic in enumerate(nmf.components_):
        top_palavras_indice = topic.argsort()[-5:][::-1]
        palavras_chave.append([vectorizer.get_feature_names_out()[indice] for indice in top_palavras_indice])

    return palavras_chave

In [7]:
# Função para extrair os cinco tópicos mais comuns para cada agência
def extrair_topicos_comuns_por_agencia(planilha):
    topicos_agencia = []

    for agencia in planilha['agencia'].unique():
        # Filtrar a planilha para a agência específica
        df_agencia = planilha[planilha['agencia'] == agencia]

        # Remover linhas com valores NaN na coluna 'comentario'
        df_agencia = df_agencia.dropna(subset=['comentario'])

        # Verificar se 'comentario' está presente nas colunas
        if 'comentario' in df_agencia.columns:
            # Garantir que 'comentario' seja uma string
            df_agencia['comentario'] = df_agencia['comentario'].astype(str)

            # Remover stopwords da coluna 'comentario'
            df_agencia['comentario'] = df_agencia['comentario'].apply(remover_stopwords)
            
            # Extrair os cinco tópicos mais comuns para a agência
            topicos = extrair_topicos_comuns(df_agencia['comentario'])
            topicos_agencia.append({'agencia': agencia, 'topicos': topicos})

        else:
            print(f"A coluna 'comentario' não foi encontrada para a agência {agencia}.")

    return topicos_agencia


In [9]:
# Carregar a planilha
planilha_path = 'C:/Users/cmarc/Documents/Faculdade/2023-02/Mineiração/Resultado Pesquisa Satisfação_Atualizada3.xlsx'
planilha = pd.read_excel(planilha_path)

# Remover linhas com valores NaN na coluna 'comentario'
planilha = planilha.dropna(subset=['comentario'])


In [18]:
# Verificar se 'comentario' está presente nas colunas
if 'comentario' in planilha.columns:
    # Garantir que 'comentario' seja uma string
    planilha['comentario'] = planilha['comentario'].astype(str)

    # Remover stopwords da coluna 'comentario'
    planilha['comentario'] = planilha['comentario'].apply(remover_stopwords)
    
    # Aplicar a análise de sentimentos à coluna 'comentario'
    planilha[['sentimento', 'probabilidade']] = pd.DataFrame(planilha['comentario'].apply(analisar_sentimento).tolist(), index=planilha.index)

    # Extrair os tópicos comuns para cada categoria única na coluna 'agencia'
    topicos_agencia = extrair_topicos_comuns_por_agencia(planilha)

    # Criar uma nova planilha com os tópicos por agência
    df_topicos_agencia = pd.DataFrame(topicos_agencia)
    df_resultado = pd.concat([planilha[['agencia', 'comentario', 'sentimento', 'probabilidade']]], axis=1)

    # Salvar a nova planilha com sentimentos
    df_resultado.to_excel('planilha_com_sentimentos_e_topicos_bert.xlsx', index=False)

    # Salvar os tópicos comuns em uma planilha separada
    df_topicos_agencia.to_excel('planilha_com_topicos_agencia.xlsx', index=False)

else:
    print("A coluna 'comentario' não foi encontrada na planilha.")